# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from tqdm import tqdm
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "is_healthy.csv"), index_col=[0, 1])
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-11  SAMEA3136645        0.000000                     0.000000   
         SAMEA3136644        0.015974                     0.015974   
         SAMEA3136672        0.000000                     0.000000   
         SAMEA3136637        0.000000                     0.000000   
         SAMEA3136639        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-11  SAMEA3136645                                            0.000000   
         SAMEA3136644                                            0.015974   
         SAMEA3136672                                            0.000000   
         SAMEA3136637                                            0.000000   
         SAMEA3136639                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-11  SAMEA3136645                                               0.000000                      
         SAMEA3136644                                               0.015974                      
         SAMEA3136672                                               0.000000                      
         SAMEA3136637                                               0.000000                      
         SAMEA3136639                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-11  SAMEA3136645                                               0.000000                                             
         SAMEA3136644                                               0.015974                                             
         SAMEA3136672                                               0.000000                                             
         SAMEA3136637                                               0.000000                                             
         SAMEA3136639                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-11  SAMEA3136645                                               0.000000                                                                   
         SAMEA3136644                                               0.015974                                                                   
         SAMEA3136672                                               0.000000                                                                   
         SAMEA3136637                                               0.000000                                                                   
         SAMEA3136639                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-11  SAMEA3136645                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-11  SAMEA3136645                              True
         SAMEA3136644                              True
         SAMEA3136672                              True
         SAMEA3136637                              True
         SAMEA3136639                              True

# Generate group splits

In [3]:
# indentify groups to partition dataset

sample_studies = np.array(X.index.get_level_values(0))
o = OrdinalEncoder()
groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
logo = LeaveOneGroupOut()

# Generate predictions in leave-one-study-out cross validation!

## GMHI

In [4]:
# instantiate the gmhi model
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

# generate predictions for each left out study (doing this manually because GMHI class required pandas dataframes)
GMHI_scores_cv = np.zeros(y.shape)
for test_study in tqdm(np.unique(X.index.get_level_values(0))):
    train_studies = ~X_GMHI.index.get_level_values(0).isin([test_study])
    X_train = X_GMHI.loc[train_studies]
    y_train = y.loc[train_studies]
    X_test = X_GMHI.loc[test_study]
    gmhi.fit(X_train, y_train)
    GMHI_scores_cv[X.index.get_level_values(0) == test_study] = np.expand_dims(gmhi.decision_function(X_test), -1)
    
GMHI_scores_cv = pd.DataFrame(GMHI_scores_cv, index=y.index, columns=["GMHI_cv"])

100%|███████████████████████████████████████████████████████████████| 53/53 [00:09<00:00,  5.51it/s]


In [5]:
idx = (abs(GMHI_scores_cv) > 0.1).values
print(balanced_accuracy_score(y, GMHI_scores_cv > 0))
print(balanced_accuracy_score(y[idx], GMHI_scores_cv[idx] > 0))

0.6987508612561699
0.7029475337112829


## GMHI2

In [6]:
# instantiate the logistic regression classifier (gmhi2)
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

# generate logit predictions for each left out study
GMHI2_scores_cv = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
    groups=groups, cv=logo, verbose=2, n_jobs=-1
)

GMHI2_scores_cv = pd.DataFrame(GMHI2_scores_cv, index=y.index, columns=["GMHI2_cv"])
GMHI2_scores_cv

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  49 out of  53 | elapsed:   16.4s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:   16.8s finished


GMHI2_cv
Study_ID Sample Accession          
GMHI-11  SAMEA3136645      0.634851
         SAMEA3136644      2.210863
         SAMEA3136672      1.083878
         SAMEA3136637     -0.644752
         SAMEA3136639      0.058963
...                             ...
P98      SAMEA6104852      0.607271
         SAMEA6104851     -1.174079
         SAMEA6104850      0.710293
         SAMEA6104885     -0.218025
         SAMEA6104858      0.953184

[8767 rows x 1 columns]

In [10]:
idx = (abs(GMHI2_scores_cv) > 0.1).values
print(balanced_accuracy_score(y, GMHI2_scores_cv > 0))
print(balanced_accuracy_score(y[idx], GMHI2_scores_cv[idx] > 0))

0.7415736335465837
0.7541776255571229


# Save cv scores

In [8]:
GMHI_scores_cv.to_csv(os.path.join(config.PREDICTION_DIR, "GMHI_scores_cv.csv"))
GMHI2_scores_cv.to_csv(os.path.join(config.PREDICTION_DIR, "GMHI2_scores_cv.csv"))